# Python и машинное обучение: нейронные сети и компьютерное зрение

## Модуль 2A. Полносвязная нейросеть



In [ ]:
!pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from torchinfo import summary

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = "cuda" if torch.cuda.is_available() else \
    "mps" if torch.backends.mps.is_built() else "cpu"
device

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

print(X.shape)
print(X.max(), X.min())
print(y[:12])
print(np.unique(y))
print(np.unique(y).shape)
print(X[0].reshape(8,8))

In [ ]:
X_tn = torch.tensor(X * 1.0/(X.max()-X.min()), 
                    device=device, 
                    dtype=torch.float32
                   )
y_tn = torch.tensor(y, device=device)
X_tn

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_tn, y_tn, 
                                                    test_size=0.3, 
                                                    random_state=20231110,
                                                   stratify = y)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_train.device)

In [ ]:
# функция для расчета точности
def accuracy_fn(logps, labels):
    pred_classes = torch.argmax(torch.exp(logps), axis=1)
    val_classes = labels
    return float(torch.eq(pred_classes, val_classes).sum() / labels.shape[0])

# функция для обучения
def train(model, epochs=10000, report_positions=20):

    results = {'epoch_count': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    # прогоняем данные по нейросети
    for epoch in range(epochs):

        # Put the model in training mode
        model.train()

        y_logps = model(X_train) #логарифмы вероятности отнесения к классам
        loss = criterion(y_logps, y_train) #кросс-энтропия

        acc = accuracy_fn(y_logps, y_train) # calculate the accuracy; convert the labels to integers

        optimizer.zero_grad() # reset the gradients so they don't accumulate each iteration
        loss.backward() # backward pass: backpropagate the prediction loss
        optimizer.step() # gradient descent: adjust the parameters by the gradients collected in the backward pass

        # Put the model in evaluation mode
        model.eval()

        with torch.inference_mode():
            y_val_logps = model(X_val)

            valid_loss = criterion(y_val_logps, y_val)
            valid_acc = accuracy_fn(y_val_logps, y_val)

        # Print progress a total of 20 times
        if epoch % (epochs // report_positions) == 0 or epochs<50:
            print(f'Epoch: {epoch+1:4.0f} | Train Loss: {loss:.5f}, Accuracy: {acc*100:.2f}% | \
                Validation Loss: {valid_loss:.5f}, Accuracy: {valid_acc*100:.2f}%')

            results['epoch_count'] += [epoch]
            results['train_loss'] += [loss.cpu().detach().numpy()]
            results['train_acc'] += [acc]
            results['val_loss'] += [valid_loss.cpu().detach().numpy()]
            results['val_acc'] += [valid_acc]

    return results

# рисовалка графиков
def plot_results(results):

    fig, axs = plt.subplots(1,2)

    fig.set_size_inches(10,3)

    for i, loss_acc in enumerate(['loss', 'acc']):
        for train_val in ['train', 'val']:
            axs[i].plot(results['epoch_count'], results[f'{train_val}_{loss_acc}'], label=f'{loss_acc} {train_val}')

        axs[i].legend()

    plt.show()

Сделаем класс для модели с возможностью вариативности по следующим параметрам:
- кол-во нейронов в скрытом слое
- функции активации

In [ ]:
class MLPDigits_vary(nn.Module):
    
    def __init__(self, activation='sigmoid', hidden=52, **kwargs):
        super().__init__()
        self.fc1 = nn.Linear(8*8, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, 10)
        self.activation = eval(f'F.{activation}')

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
vary = 'activation' #здесь пишем, что варьируем
var_values = ['relu', 'tanh', 'sigmoid'] # здесь перечисляем варианты


dict_vary = {'hidden': 52,
            'activation': 'tanh',
            'lr': 0.2,
            'momentum': 0.9,
            'optimizer': 'SGD',
            'epochs': 100}

dict_acc = {} # here we collect data for comparison

criterion = nn.CrossEntropyLoss()

for var in var_values:
    print(f"{vary}: {var}")
    dict_vary[vary] = var
    model = MLPDigits_vary( **dict_vary ).to(device)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum']) \
            if dict_vary['optimizer']=='RMSprop' else \
                torch.optim.Adam(model.parameters(), lr=dict_vary['lr']) \
            if dict_vary['optimizer']=='Adam' else \
                    torch.optim.SGD(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum'])
    results = train(model, epochs=dict_vary['epochs'], report_positions=10)
    dict_acc[f'err_{var}'] = results['val_loss']
    dict_acc[f'acc_{var}'] = results['val_acc']
    dict_acc[f'epochs_{var}'] = results['epoch_count']

    plot_results(results)


fig_var, axs_var = plt.subplots(1,2)
fig_var.set_size_inches(10,3)
for var in var_values:
    axs_var[0].plot(dict_acc[f'epochs_{var}'], dict_acc[f'err_{var}'], label=f'loss on {vary}={var}')
    axs_var[0].legend()
    axs_var[1].plot(dict_acc[f'epochs_{var}'], dict_acc[f'acc_{var}'], label=f'acc on {vary}={var}')
    axs_var[1].legend()


plt.show()

#### Практика

Используя код выше, поэкспериментируйте с моделью:
 - попробуйте изменить количество весов в скрытом и выходном слоях (попробуйте 50, 32, 16, 8, 2, 1);
 - попробуйте различную скорость обучения;
 - попробуйте различные функции активации;
 - поэкпериметируйте с количеством эпох.
 
Постарайтесь выйти на оптимальный уровень точности и скорости обучения.
 
Сообщите максимальное значение точности на валидационном подмножестве данного датасета, конфигурацию модели и параметры обучения.


In [ ]:
%%time
dict_vary = ? # ваши параметры настройки модели

model = MLPDigits_vary( **dict_vary ).to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum']) \
        if dict_vary['optimizer']=='RMSprop' else \
            torch.optim.Adam(model.parameters(), lr=dict_vary['lr']) \
        if dict_vary['optimizer']=='Adam' else \
                torch.optim.SGD(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum'])
results = train(model, epochs=dict_vary['epochs'], report_positions=10)


### Загрузка данных в модель в виде мини-батчей

Выше мы обучали модель на всех имеющихся данных. Но при обучении нейросетей данные чаще всего загружают в виде мини-пакетов (мини-батчей). Это делается для того, чтобы:
- обучать модель на разнообразии данных: в рамках одной эпохи каждый цикл обучения она будет видеть уникальные данные;
- минимизировать объем памяти, используемой при обучении;
- ускорить работу оптимизатора: он будет гораздо быстрее обрабатывать небольшой объем данных6 нежели чем весь датасет.

Для этого можно использовать встроенный в PyTorch механизм генерации мини-батчей. На базе стандартного генератора PyTorch можно создать свой собственный, который будет отправлять в модель именно ваши данные в нужном именно вам виде.

Для начала нужно создать свой класс на базе класса PyTorch ```Dataset```:

```python
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])
````

Затем на базе этого класса будут созданы объекты-генераторы данных, уже на этапе создания в них будет передана функция ```transforms.ToTensor()``` для преобразования изображений "на лету", по требованию.

In [ ]:
# класс генератора:
class DatasetDigits(Dataset):
    
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        # сразу представляем цифру как ndarray размерностью (Height * Width * Channels)
        # конвертируем цифры в np.uint8 [Unsigned integer (0 to 255)] - стандарт для изображений
        # чтобы отрабатывала стандартная функция ToTensor(), мы определяем размерность тензора (H, W, C)
        image = self.X[index].astype(np.uint8).reshape((8, 8, 1))
        
        if self.transform is not None:
            image = self.transform(image)
            
        if self.y is not None:
            return (image, self.y[index])  
        else:
            return image

# снова сделаем разбиение на обучающую и валидационную выборки
# старые переменные X_train, X_val... - это нормированные тензоры, а нам нужны изображения в исходном формате
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=20231110,
                                                   stratify = y)



# создадим генераторы обучающих и тестовых данных:
train_data = DatasetDigits(X_train, y_train, transform=transforms.ToTensor())
val_data = DatasetDigits(X_val, y_val, transform=transforms.ToTensor())

In [ ]:
batch_size = 20

train_generator = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=SubsetRandomSampler(list(range(y_train.shape[0]))))
val_generator=  torch.utils.data.DataLoader(val_data, batch_size=batch_size,
    sampler=SubsetRandomSampler(list(range(y_val.shape[0]))))


In [ ]:
def train_batches(model, 
                  train_generator,
                  valid_generator,
                  batch_size=20, epochs=40, report_positions=20, **kwargs):
    
    results = {'epoch_count': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    # прогоняем данные по нейросети
    for epoch in range(epochs):
        model.train()
        
        train_loss = valid_loss = 0.0; 
        train_correct = valid_correct = 0.0
        
        for X_batch, y_batch in train_generator:
            
            X_batch = X_batch.to(device); y_batch = y_batch.to(device)
            
            y_logps = model(X_batch) #логарифмы вероятности отнесения к классам
            loss = criterion(y_logps, y_batch) #кросс-энтропия
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.data.item()
            train_correct += accuracy_fn(y_logps, y_batch) * y_batch.shape[0]
            
        train_loss /= len(train_generator.dataset)
        train_acc = 100 * train_correct / len(train_generator.dataset)

        # Валидацию тоже делаем по батчам
        model.eval()         
        
        for valid_batches, (X_val_batch, y_val_batch) in enumerate(valid_generator):
            X_val_batch = X_val_batch.to(device); y_val_batch = y_val_batch.to(device)
            y_batch_logps = model(X_val_batch)
            loss = criterion(y_batch_logps, y_val_batch)
            
            valid_loss += loss.data.item()
            valid_correct += accuracy_fn(y_batch_logps, y_val_batch) * y_val_batch.shape[0]
            
        valid_loss /= len(valid_generator.dataset)
        valid_acc = 100 * valid_correct / len(valid_generator.dataset)
        
        results['epoch_count'] += [epoch]
        results['train_loss'] += [ train_loss ]
        results['train_acc'] += [ train_acc ]
        results['val_loss'] += [ valid_loss ]
        results['val_acc'] += [ valid_acc ]
        
        if epoch % (epochs // report_positions) == 0 or epochs<50:
            print(f"Epoch: {epoch+1:4.0f} | Train Loss: {train_loss:.5f}, "+\
                  f"Accuracy: {train_acc:.2f}% | \
                Validation Loss: {valid_loss:.5f}, Accuracy: {valid_acc:.2f}%")
            
    return results

Будем обучать ту же модель, что и в прошлый раз, ```MLPDigits_vary()```, только добавим ей "выпрямляющий" код на входе, чтобы она могла работать с тензорами-изображениями PyTorch: 

In [ ]:
class MLPDigits_vary(nn.Module):
    
    def __init__(self, activation='sigmoid', hidden=52, **kwargs):
        super().__init__()
        self.fc1 = nn.Linear(8*8, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, 10)
        self.activation = eval(f'F.{activation}')

    def forward(self, x):
        x = x.view(-1, 8 * 8) # изображение приходит в формате (1,8,8), делаем его плоским
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

criterion = nn.CrossEntropyLoss()


Поэкспериментируйте с моделью, варьируйте количество эпох и размер пакета.

In [ ]:
class MLPDigits_vary(nn.Module):
    
    def __init__(self, activation='sigmoid', hidden=52, **kwargs):
        super().__init__()
        self.fc1 = nn.Linear(8*8, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, 10)
        self.activation = eval(f'F.{activation}')

    def forward(self, x):
        x = x.view(-1, 8 * 8) # изображение приходит в формате (1,8,8), делаем его плоским
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

criterion = nn.CrossEntropyLoss()

In [ ]:
vary = 'batch_size' #здесь пишем, что варьируем
var_values = [64, 25, 10] # здесь перечисляем варианты


dict_vary = {'hidden': 12,
            'activation': 'tanh',
            'batch_size': 20,
            'lr': 0.002,
            'momentum': 0.9,
            'optimizer': 'RMSprop',
            'epochs': 30}

dict_acc = {} # here we collect data for comparison

for var in var_values:
    print(f"{vary}: {var}")
    dict_vary[vary] = var
    model = MLPDigits_vary( **dict_vary ).to(device)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum']) \
            if dict_vary['optimizer']=='RMSprop' else \
                torch.optim.Adam(model.parameters(), lr=dict_vary['lr']) \
            if dict_vary['optimizer']=='Adam' else \
                    torch.optim.SGD(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum'])
    
    # добавляем создание генератора для нужного нам количества батчей
    train_generator = torch.utils.data.DataLoader(train_data, 
                                                  batch_size=dict_vary['batch_size'],
        sampler=SubsetRandomSampler(list(range(y_train.shape[0]))))
    valid_generator = torch.utils.data.DataLoader(val_data, 
                                                  batch_size=dict_vary['batch_size'],
        sampler=SubsetRandomSampler(list(range(y_val.shape[0]))))
    
    results = train_batches(model, train_generator, valid_generator, report_positions=10, **dict_vary)
    dict_acc[f'err_{var}'] = results['val_loss']
    dict_acc[f'acc_{var}'] = results['val_acc']
    dict_acc[f'epochs_{var}'] = results['epoch_count']
    
    plot_results(results)

    
fig_var, axs_var = plt.subplots(1,2)
fig_var.set_size_inches(10,3)
for var in var_values:
    axs_var[0].plot(dict_acc[f'epochs_{var}'], dict_acc[f'err_{var}'], label=f'loss on {vary}={var}')
    axs_var[0].legend()
    axs_var[1].plot(dict_acc[f'epochs_{var}'], dict_acc[f'acc_{var}'], label=f'acc on {vary}={var}')
    axs_var[1].legend()
    
    
plt.show()

summary(model, 
        input_size=X_train.shape, 
        col_names=["input_size", "output_size", "num_params"],
        device=device
       )

## MNIST

Попробуем теперь обучить нашу модель на датасете MNIST, это те же "рукописные цифры", но в разрешении 28x28 и в количестве 50000.

In [ ]:
from torchvision import datasets

trainset = datasets.MNIST('./data', download=True, train=True, transform=transforms.ToTensor())
valset = datasets.MNIST('./data', download=True, train=False, transform=transforms.ToTensor())

print(len(trainset))
print(len(valset))

In [ ]:
# параметры нормализации
imgs = torch.stack([img for img, _ in trainset], dim=0)

mean = imgs.view(1, -1).mean(dim=1)    # or imgs.mean()
std = imgs.view(1, -1).std(dim=1)     # or imgs.std()
mean, std

In [ ]:
mnist_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])

trainset = datasets.MNIST('./data', download=True, train=True, transform=mnist_transforms)
valset = datasets.MNIST('./data', download=True, train=False, transform=mnist_transforms)

In [ ]:
batch_size=128

train_generator_MNIST = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_generator_MNIST = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)

In [ ]:
# визуализируем батч
dataiter = iter(train_generator_MNIST)
images, labels = next(dataiter)
images = images.numpy()

print(images.shape)

fig = plt.figure(figsize=(20, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

Cоздадим модель на базе последней, повысим размерность входного слоя до 28x28:

In [ ]:
class MLP_MNIST_vary(nn.Module):
    
    def __init__(self, activation='sigmoid', hidden=52, **kwargs):
        super().__init__()
        self.fc1 = nn.Linear(28*28, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, 10)
        self.activation = eval(f'F.{activation}')

    def forward(self, x):
        x = x.view(-1, 28 * 28) # изображение приходит в формате (1,8,8), делаем его плоским
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)



In [ ]:
dict_vary = {'hidden': 512,
            'activation': 'tanh',
            'batch_size': batch_size,
            'lr': 0.01,
            'momentum': 0.9,
            'optimizer': 'RMSprop',
            'epochs': 10}

model = MLP_MNIST_vary( **dict_vary ).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=dict_vary['lr'], momentum=dict_vary['momentum']) 
criterion = nn.CrossEntropyLoss()

print(model)

results = train_batches(model, 
                        train_generator_MNIST, 
                        val_generator_MNIST, report_positions=10, **dict_vary)

plot_results(results)
summary(model, 
        input_size=images.shape, 
        col_names=["input_size", "output_size", "num_params"],
        device=device
       )

## Регуляризация добавлением слоя Dropout

Добавим в модель "прореживание" - слой dropout, который в момент обучения блокирует ряд нейронов (позиций в матрицах соотв. слоев).